In [1]:
%matplotlib inline 

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import os
import time
import xarray as xr

#### PATHS- set for multiple computers, see bottom to SET PATH

In [2]:
#ASO lidar - netCDFs & integers in [mm]
ars_path = '/home/meganmason/Documents/projects/thesis/data/processing_lidar/depths_3m/equal_extent_data_downsize/nc/20*/*int.nc'
bsu_path = '/Users/meganmason491/Documents/research/sierra/data/mm/20*/*int.nc'
sonic_path = '/home/meganmason/Research/Sierra/data/mm/20*/*int.nc'

#DEM & Mask for Tuolumne (topo.nc @ 3m cell size) - dem here is from national database...(10m>3m)
ars_topo_path = '/home/meganmason/Documents/projects/thesis/maps/map_layers/basin_ops/tuolumne/topo/basin_setup_3m/topo.nc'
bsu_topo_path = '/Users/meganmason491/Documents/research/sierra/data/topo.nc'
sonic_topo_path = '/home/meganmason/Research/Sierra/data/topo.nc'

# ASO snow-free and surface class
ars_terrain_path = '/home/meganmason/Documents/projects/thesis/data/processing_lidar/depths_3m/equal_extent/*.nc'
bsu_terrain_path = '/Users/meganmason491/Documents/research/sierra/data/terrain/*.nc'
# sonic_terrain_path = ''


# ~~ SET PATH HERE ~~ #
path = bsu_path
topo_path = bsu_topo_path
terrain_path = bsu_terrain_path

#### FILEPATH LIST AND DATE PARSING (lidar only here)

In [3]:
# filepath list, lidar
flist = sorted(glob.glob(path))
print('# of lidar files as input:', len(flist))

# Parse dates, store as pandas datetime type in list
fparse = [d.split('/')[-1] for d in flist] #splits fname on '/' saves last
fparse = [d.split('_')[0] for d in fparse] #splits fname on '_' saves first

# flight dates, parsed from filename
dates = pd.to_datetime(fparse) #pandas datetime variable, lidar dates
print('dates, file type is:', type(dates))

# of lidar files as input: 51
dates, file type is: <class 'pandas.core.indexes.datetimes.DatetimeIndex'>


### OPTION: run a smaller chunk, peak, etc.
Activate if you want to run 2 flights, peak flights, etc.

In [4]:
# '''first 2 files'''
# # flist = flist[:2] #first 6 are just 2013
# # dates = dates[:2]

# '''peak snow depth dates....(?)'''
# flist = list(flist[i] for i in (0,7,22,28,41,49))
# dates = list(dates[i] for i in (0,7,22,28,41,49))

# print('# of lidar files as input:', len(flist))
# print('# of flight dates', len(dates))

#### BUILD XARRAY DATASET
* open multiple files (mf) from file list (lidar res = 3m)
* name the concatinating dimention, 'time'
* 'nested' vs 'comb_by_cords' .....not totally clear, but no error!
* set chunk size (prefered is 1 million in size (1000x1000)
* activate DASK with parallel
* rename data to 'snow' on the fly
* drop a variable
* close to remove from memory

#### ADD 'TIME' COORDINATE 

* time coord from file name dates, by default populates with 'time' dimension

#### FLIP 'Y' COORD (corrects orientation for imshow() ) 
* flips horizonally (but actually flips 'Y')
    *NOTE: tried several options flipud, flip w/ axis=0, etc. 
* flip causes DataSet to become a DataArray, set back to DataSet
    *NOTE: changes set chunk size, solution - set chunk size when compiled netCDF is loaded in analysis

In [5]:
# build xarray
ds=xr.open_mfdataset(flist, 
                     concat_dim='time', 
                     combine='nested',
                     chunks={'x':1000, 'y':1000}, 
                     parallel=True).rename({'Band1':'snow'}).drop('transverse_mercator') #combine='nested', 

ds.close()

# add 'time' coord
ds['time'] = (['time'], dates) 

# flip 'y' up/down
ds=np.flip(ds.snow,1) 
ds=ds.to_dataset()

ds

<xarray.Dataset>
Dimensions:  (time: 51, x: 17569, y: 17002)
Coordinates:
  * y        (y) float64 4.23e+06 4.23e+06 4.23e+06 ... 4.179e+06 4.179e+06
  * x        (x) float64 2.54e+05 2.54e+05 2.54e+05 ... 3.067e+05 3.067e+05
  * time     (time) datetime64[ns] 2013-04-03 2013-04-29 ... 2018-05-28
Data variables:
    snow     (time, y, x) float32 dask.array<chunksize=(1, 2, 1000), meta=np.ndarray>

#### Open topo.nc (dem, mask, veg_height)
Cell size is 3m

In [6]:
topo = xr.open_dataset(topo_path)#, drop_variables=['veg_tau','veg_k','veg_type','projection'], chunks={'x':1000, 'y':1000})
topo.close()

In [7]:

# 1). assign mask to ds coords
ds = ds.assign_coords(mask=topo.mask)

# 2). assign 'dem' and 'veg' as coords where mask==1 
ds = ds.assign_coords(dem=topo.dem.where(ds.mask==1))
ds = ds.assign_coords(veg=topo.veg_height.where(ds.mask==1))

# 3). mask the watershed (snow is only variable in ds)
ds=ds.where(ds.mask==1)

# 4). change dtypes of snow depth before saving output
ds=ds.astype(np.int16, copy=False)


#### Concatenate topo layers with lidar DataSet (ds)

In [8]:
# add topo features to lidar DataSet (ds) as DATA_VARS
# ds['mask'] = (['y','x'],topo.mask) 
# ds['dem'] = (['y','x'],topo.dem)
# ds['veg'] = (['y','x'],topo.veg_height)

#### SHOW THAT MASK IS APPLIED TO DATASET
~ Do some subplots here to show this nicer.... ~

In [9]:
plt.imshow(ds.snow[0]/1000, cmap='coolwarm'); plt.colorbar(); plt.title('One snow depth image'); plt.show() #zeros outside of mask because nans=float, 0=ints
plt.imshow(ds.dem, cmap='terrain'); plt.colorbar(); plt.title('DEM'); plt.show()
plt.imshow(ds.veg, cmap='Greens'); plt.colorbar(); plt.title('Veg Height'); plt.show()

In [10]:
# plt.imshow(ds.snow[0]/1000, cmap='coolwarm'); plt.colorbar(); plt.title('One snow depth image'); plt.show() #zeros outside of mask because nans=float, 0=ints
fig=plt.figure(figsize=(8,8))
plt.imshow(ds.dem, cmap='terrain'); plt.title('DEM'); plt.axis('off')
plt.savefig('../figs/dem_for_graphic', dpi=300, transparent=True)

fig=plt.figure(figsize=(8,8))
plt.imshow(ds.veg, cmap='summer'); plt.title('Veg Height'); plt.axis('off')
plt.savefig('../figs/vegHeight_for_graphic', dpi=300, transparent=True)

#### SAVE OUTPUT

In [11]:
ds

<xarray.Dataset>
Dimensions:  (time: 51, x: 17569, y: 17002)
Coordinates:
  * y        (y) float32 4230327.0 4230324.0 4230321.0 ... 4179327.0 4179324.0
  * x        (x) float32 254007.0 254010.0 254013.0 ... 306708.0 306711.0
  * time     (time) datetime64[ns] 2013-04-03 2013-04-29 ... 2018-05-28
    mask     (y, x) uint8 0 0 0 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0 0 0
    dem      (y, x) float32 nan nan nan nan nan nan ... nan nan nan nan nan nan
    veg      (y, x) float32 nan nan nan nan nan nan ... nan nan nan nan nan nan
Data variables:
    snow     (time, y, x) int16 dask.array<chunksize=(1, 2, 1000), meta=np.ndarray>

In [12]:
#uncomment when you want to save:
# ds.to_netcdf('~/Documents/projects/thesis/results/output/compiled_SUPERsnow.nc')